<a href="https://colab.research.google.com/github/SubhankarSamanta295/IMDB-sentiment-analysis/blob/main/NLP_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
!pip install -U spacy
!python -m spacy download en_core_web_lg

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.1.0)
2021-07-15 10:23:58.381236: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
data = open('/content/drive/MyDrive/sentiment labelled sentences/imdb_labelled.txt')

In [4]:
text = data.read()

In [5]:
import re
sent=re.findall(r'.*\n',text)

In [6]:
sent=[(line[:-3],int(line[-2])) for line in sent]

In [7]:
data=pd.DataFrame(sent,columns=['review','score'])

In [8]:
data.head()

,review,score
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [9]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data['review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [11]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
x = cv.fit_transform(corpus).toarray()

In [12]:
y = data['score']

In [13]:
# Train Test Split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [14]:
# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
classifier_multi = MultinomialNB(alpha = 2.2)
classifier_multi.fit(x_train, y_train)

MultinomialNB(alpha=2.2, class_prior=None, fit_prior=True)

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

#Visualizing the performace of the model
y_pred_test = classifier_multi.predict(x_test)

#Printing the classification_report
print(classification_report(y_test, y_pred_test))

#Printing the confusion_matrix
cm = confusion_matrix(y_test , y_pred_test)
print(cm)

#Printing the accuracy score
acc_score = accuracy_score(y_pred_test , y_test)
acc_score = acc_score*100
print('Accuracy of testing set {}%'.format(acc_score))

              precision    recall  f1-score   support

           0       0.83      0.84      0.83       140
           1       0.86      0.85      0.85       160

    accuracy                           0.84       300
   macro avg       0.84      0.84      0.84       300
weighted avg       0.84      0.84      0.84       300

[[117  23]
 [ 24 136]]
Accuracy of testing set 84.33333333333334%
